# Using tool to get information about a country, from wikipedia




## Import libaries and prepare the API credential

In [7]:
from autogen import ConversableAgent, register_function

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## A Python function to call Wiki API


## Agent 1: tool executor (just to wrap up the function)

In [ ]:

wiki_tool = ConversableAgent(
    "wiki_tool",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    # is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

### Agent 2: Tool driver (caller of the function)

In [10]:
wiki_tool_driver = ConversableAgent(
    "wiki_tool_driver",
    system_message = "You are a helpful assistant. You may decide to use some external tools if necessary"
    "If you are happy with the result of the tool, just say TERMINATE",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

### Register the tool to both caller and executor

*Question:* Why we need tool agents to use a tool?

In [11]:
from tools.tools import describe_country

register_function(
    describe_country,
    caller = wiki_tool_driver,
    executor = wiki_tool,
    description = "get the description of a country from wiki. this service only receive the formal name of a country"
)

## Now we start

In [12]:
wiki_tool.initiate_chat(
    wiki_tool_driver,
    message = "I want to know something about a country with nick name uncle sam",
    max_turns = 2
)


wiki_tool (to wiki_tool_driver):

I want to know something about a country with nick name uncle sam

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
wiki_tool_driver (to wiki_tool):

***** Suggested tool call (call_GUgkGpyazEMwnbYf8D1No4xx): describe_country *****
Arguments: 
{"country_name":"United States of America"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION describe_country...
wiki_tool (to wiki_tool_driver):

wiki_tool (to wiki_tool_driver):

***** Response from calling tool (call_GUgkGpyazEMwnbYf8D1No4xx) *****
The United States of America (USA or U.S.A.), commonly known as the United States (US or U.S.) or America, is a country primarily located in North America. It is a federation of 50 states, which also includes its federal capital distr

ChatResult(chat_id=None, chat_history=[{'content': 'I want to know something about a country with nick name uncle sam', 'role': 'assistant'}, {'tool_calls': [{'id': 'call_GUgkGpyazEMwnbYf8D1No4xx', 'function': {'arguments': '{"country_name":"United States of America"}', 'name': 'describe_country'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': "The United States of America (USA or U.S.A.), commonly known as the United States (US or U.S.) or America, is a country primarily located in North America. It is a federation of 50 states, which also includes its federal capital district of Washington, D.C., and 326 Indian reservations. The 48 contiguous states border Canada to the north and Mexico to the south. The State of Alaska is non-contiguous and lies to the northwest, while the State of Hawaii is an archipelago in the Pacific Ocean. Outside the union of states, the U.S. also asserts sovereignty over five major unincorporated island territories and various uninh

## Exercise
1. Try to register multiple tools in one agent, and let the drive decide which tool to call
2. Combine code executation and tool usage: Create a tool agent that can query a Neo4j database, and the driver need to generate the CYPHER query and call the tool.